In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
from keras.datasets import mnist
import sys
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, SpatialDropout2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K

from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        Mean of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 10))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    prediction = result.mean(axis=0)
    return prediction

def predict_pool_with_uncertainty(f, x, n_iter=50):
    """Function generating and returning non-deterministic predictions using MC dropout
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        All MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 10))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    return result


def run_model (train_data_indices, pool_sample_indices):
  """Initializes and trains a Keras model from scratch on the given training data, returning test MAE and MSE (and MC dropout predictions for D_{sample})
  #Arguments
    train_data_indices: indices within X_train_All of current training points
    pool_sample_indices: indices within X_train_All of pool points in the selected sample D_{sample}
  #Returns
    Test Accuracy and MC dropout predictions for all pool points in D_{sample}
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train = keras.utils.to_categorical(y_train, num_classes=10)
  train_size = y_train.shape[0]
  Weight_Decay = 0.01/train_size
  dropout_prob = 0.25
  nb_filters = 40
  nb_pool = 3
  nb_conv = 4
  img_rows = img_cols = 28
  nb_classes = 10
  model = Sequential()
  model.add(Convolution2D(nb_filters, kernel_size=nb_conv, strides=1, data_format="channels_first", input_shape=(1, img_rows, img_cols)))
  model.add(Activation('relu'))
  model.add(Convolution2D(nb_filters, kernel_size=nb_conv, strides=2, data_format = "channels_first"))
  model.add(Activation('relu'))
	
  model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
  model.add(Dropout(dropout_prob))

  model.add(Flatten())
  model.add(Dense(128, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('relu'))
  model.add(Dropout(dropout_prob))
  model.add(Dense(nb_classes, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam')
  model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=0)
  f = K.function([model.layers[0].input,K.learning_phase()],[model.layers[-1].output])
  y_test_output = predict_with_uncertainty(f, X_test, n_iter=100)
  y_test_predictions = np.argmax(y_test_output, axis=1)
  pool_sample_predictions = predict_pool_with_uncertainty(f, np.expand_dims(X_train_All[pool_sample_indices], axis=1), n_iter=50)
  return [np.sum(y_test_predictions==y_test_original)/(y_test_original.shape[0]), pool_sample_predictions]


def get_acquisition_fn(pool_forward_pass, batch_size):
  """Evaluates Max Entropy acquisition function (pool_forward_pass shape is [num_masks (J), number of pool candidate points, number of classes])
  #Arguments:
      pool_forward_pass: MC dropout predictions on candidate points for acquisition in the pool
      batch_size: size of queried batch
  #Returns:
      Indices of points in queried batch (where points are ordered as in pool_forward_pass)
  """
  average_predictions = np.mean(pool_forward_pass, axis=0)
  all_entropies = entropy(average_predictions, axis=1) 
  return np.flip(np.argsort(all_entropies))[0:batch_size]

In [0]:
#Active learning parameters/settings

batch_size = 10
dropout_prob = 0.25
num_experiments = 3
num_acquisitions = 1000
num_candidates = 2000
num_masks = 50
#Keras Model Parameters
num_classes = 10
nb_filters = 40
nb_pool = 2 
nb_conv = 4
img_rows = img_cols = 28

In [0]:
#Loading data
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
		  delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
		 delimiter=",") 
y_train_All = train_data[:,0]
y_test = test_data[:,0]
X_train_All = train_data[:,1:].reshape((60000,28,28))
X_test = test_data[:,1:].reshape((10000,28,28)) 
train_ind = np.arange(60000)
test_ind = np.arange(10000)

y_test_original = y_test
y_test = keras.utils.to_categorical(y_test, num_classes=10)
X_test = np.expand_dims(X_test[test_ind], axis=1)
folder_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/MultiClass_AL_Scripts_v1/"
out_folder_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/MultiClass_AL_Results_v1/"


print('Running')	  
all_tr_ind = []
all_acc = []
#Iterating across experiments, each of which begins with a different training set (that is balanced across classes)

for e in range(num_experiments):
  exp_acc = []
  acc_file = "MaxEntAccBS"+str(batch_size)+"Ind"+str(e+1)+".npy"
  ind_file = "MaxEntIndBS"+str(batch_size)+"Ind"+str(e+1)+".npy"
  train_data_indices = list(np.load(folder_path + 'trainindices' + str(e+1) + '.npy'))
  pool_indices = [i for i in train_ind if i not in train_data_indices]
  new_train_cand = np.random.choice(pool_indices, num_candidates, replace=False)
  #Evaluating initial model
  model_results = run_model(train_data_indices, new_train_cand)
  print('trained model')
  exp_acc.append(model_results[0])
  all_acc.append(exp_acc)
  all_tr_ind.append(train_data_indices)
  print('Initial Acc: ' + str(all_acc))
  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size):
    #Maximum entropy acquisition
    new_train_ind_ind = get_acquisition_fn(model_results[1], batch_size)
    new_train_indices = new_train_cand[new_train_ind_ind]
    train_data_indices = list(train_data_indices)+list(new_train_indices)
    pool_indices = list(set(pool_indices) - set(new_train_indices))      
    new_train_cand = np.random.choice(pool_indices, num_candidates, replace=False)
    #Evaluating model and saving results
    model_results = run_model(train_data_indices, new_train_cand)
    exp_acc.append(model_results[0])
    all_acc[-1] = exp_acc
    all_tr_ind[-1] = train_data_indices
    print('len train ind: ' + str(len(train_data_indices)))
    print('len pool ind: ' + str(len(pool_indices)))
    np.save(out_folder_path+acc_file, np.array(exp_acc))
    np.save(out_folder_path+ind_file, np.array(train_data_indices))
    print('All Acc: ' + str(all_acc))
    #print('All Ind: ' + str(all_tr_ind))
print('All Acc: ' + str(all_acc))




Running


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


trained model
Initial Acc: [[0.5244]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


len train ind: 40
len pool ind: 59960
All Acc: [[0.5244, 0.6296]]


...

len train ind: 1010
len pool ind: 58990
All Acc: [[0.5244, 0.6296, 0.634, 0.6763, 0.6927, 0.7288, 0.7593, 0.7596, 0.7614, 0.7616, 0.7875, 0.7996, 0.8283, 0.8103, 0.8528, 0.8283, 0.8218, 0.8175, 0.8436, 0.869, 0.8606, 0.8445, 0.8695, 0.8357, 0.8293, 0.8426, 0.8656, 0.8624, 0.9039, 0.8789, 0.8832, 0.9049, 0.8945, 0.8515, 0.8875, 0.9101, 0.8893, 0.8713, 0.8811, 0.8984, 0.9121, 0.8939, 0.8966, 0.9143, 0.88, 0.9166, 0.9345, 0.9088, 0.9285, 0.9285, 0.9183, 0.9043, 0.9089, 0.9343, 0.9117, 0.9101, 0.9036, 0.9166, 0.9207, 0.9123, 0.9342, 0.9086, 0.9367, 0.9128, 0.9265, 0.9483, 0.9175, 0.9205, 0.9391, 0.9022, 0.9029, 0.913, 0.9431, 0.9303, 0.9185, 0.9294, 0.9259, 0.9319, 0.9267, 0.936, 0.9388, 0.9358, 0.89, 0.9387, 0.937, 0.9415, 0.9428, 0.9162, 0.9631, 0.9453, 0.9346, 0.9325, 0.9205, 0.9326, 0.9392, 0.9387, 0.9435, 0.9504, 0.9473, 0.9439, 0.9462], [0.5524, 0.6451, 0.6953, 0.716, 0.6999, 0.7597, 0.7684, 0.8138, 0.8208, 0.8111, 0.7624, 0.8041, 0.8293, 0.8003, 0.8711, 0.8472, 0.8368, 0.8824, 0.839

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


len train ind: 1020
len pool ind: 58980
All Acc: [[0.5244, 0.6296, 0.634, 0.6763, 0.6927, 0.7288, 0.7593, 0.7596, 0.7614, 0.7616, 0.7875, 0.7996, 0.8283, 0.8103, 0.8528, 0.8283, 0.8218, 0.8175, 0.8436, 0.869, 0.8606, 0.8445, 0.8695, 0.8357, 0.8293, 0.8426, 0.8656, 0.8624, 0.9039, 0.8789, 0.8832, 0.9049, 0.8945, 0.8515, 0.8875, 0.9101, 0.8893, 0.8713, 0.8811, 0.8984, 0.9121, 0.8939, 0.8966, 0.9143, 0.88, 0.9166, 0.9345, 0.9088, 0.9285, 0.9285, 0.9183, 0.9043, 0.9089, 0.9343, 0.9117, 0.9101, 0.9036, 0.9166, 0.9207, 0.9123, 0.9342, 0.9086, 0.9367, 0.9128, 0.9265, 0.9483, 0.9175, 0.9205, 0.9391, 0.9022, 0.9029, 0.913, 0.9431, 0.9303, 0.9185, 0.9294, 0.9259, 0.9319, 0.9267, 0.936, 0.9388, 0.9358, 0.89, 0.9387, 0.937, 0.9415, 0.9428, 0.9162, 0.9631, 0.9453, 0.9346, 0.9325, 0.9205, 0.9326, 0.9392, 0.9387, 0.9435, 0.9504, 0.9473, 0.9439, 0.9462], [0.5524, 0.6451, 0.6953, 0.716, 0.6999, 0.7597, 0.7684, 0.8138, 0.8208, 0.8111, 0.7624, 0.8041, 0.8293, 0.8003, 0.8711, 0.8472, 0.8368, 0.8824, 0.839

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


len train ind: 1030
len pool ind: 58970
All Acc: [[0.5244, 0.6296, 0.634, 0.6763, 0.6927, 0.7288, 0.7593, 0.7596, 0.7614, 0.7616, 0.7875, 0.7996, 0.8283, 0.8103, 0.8528, 0.8283, 0.8218, 0.8175, 0.8436, 0.869, 0.8606, 0.8445, 0.8695, 0.8357, 0.8293, 0.8426, 0.8656, 0.8624, 0.9039, 0.8789, 0.8832, 0.9049, 0.8945, 0.8515, 0.8875, 0.9101, 0.8893, 0.8713, 0.8811, 0.8984, 0.9121, 0.8939, 0.8966, 0.9143, 0.88, 0.9166, 0.9345, 0.9088, 0.9285, 0.9285, 0.9183, 0.9043, 0.9089, 0.9343, 0.9117, 0.9101, 0.9036, 0.9166, 0.9207, 0.9123, 0.9342, 0.9086, 0.9367, 0.9128, 0.9265, 0.9483, 0.9175, 0.9205, 0.9391, 0.9022, 0.9029, 0.913, 0.9431, 0.9303, 0.9185, 0.9294, 0.9259, 0.9319, 0.9267, 0.936, 0.9388, 0.9358, 0.89, 0.9387, 0.937, 0.9415, 0.9428, 0.9162, 0.9631, 0.9453, 0.9346, 0.9325, 0.9205, 0.9326, 0.9392, 0.9387, 0.9435, 0.9504, 0.9473, 0.9439, 0.9462], [0.5524, 0.6451, 0.6953, 0.716, 0.6999, 0.7597, 0.7684, 0.8138, 0.8208, 0.8111, 0.7624, 0.8041, 0.8293, 0.8003, 0.8711, 0.8472, 0.8368, 0.8824, 0.839